In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install eli5

from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from numpy import mean, std
import eli5
from eli5.sklearn import PermutationImportance
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 6.8 MB/s 
     |████████████████████████████████| 133 kB 51.5 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=27d85fcee09d343d77750f098a237b9e85fcb81895790f0f075f4326b176d990
  Stored in directory: /root/.cache/pip/wheels/cc/3c/96/3ead31a8e6c20fc0f1a707fde2e05d49a80b1b4b30096573be
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.


In [ ]:
data = pd.read_excel('/content/drive/MyDrive/AdaBoost/Rata2 IPK per Semester - Revisi.xlsx', sheet_name='detil')
data.head()

,NIM,NAMA MAHASISWA,PROGRAM STUDI,FAKULTAS,SEMESTER,SKS,BOBOT,IPK
0,1101617001,RACHMAT HIDAYAT,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,150,539.3,3.59
1,1101617002,ARIF BUDI SANTOSO,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,148,553.9,3.74
2,1101617003,NURUL HISNA HERYANTO,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,159,571.3,3.59
3,1101617004,ALDINO RAMADHAN,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,148,546.7,3.69
4,1101617005,MUHAMMAD AZHAR THIFAL,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,158,593.2,3.75


In [ ]:
data = data[~(data['FAKULTAS'] == 'Pascasarjana')] # buang yg pasca sarjana
data

,NIM,NAMA MAHASISWA,PROGRAM STUDI,FAKULTAS,SEMESTER,SKS,BOBOT,IPK
0,1101617001,RACHMAT HIDAYAT,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,150,539.3,3.59
1,1101617002,ARIF BUDI SANTOSO,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,148,553.9,3.74
2,1101617003,NURUL HISNA HERYANTO,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,159,571.3,3.59
3,1101617004,ALDINO RAMADHAN,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,148,546.7,3.69
4,1101617005,MUHAMMAD AZHAR THIFAL,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,158,593.2,3.75
...,...,...,...,...,...,...,...,...
566400,8335165396,FAUZI AKBAR EKA SAPUTRA,17066-Akuntansi,Fakultas Ekonomi,IPK 105,20,69.9,3.49
566401,8335165418,MUHAMMAD SYARIP RANGKUTI,17066-Akuntansi,Fakultas Ekonomi,IPK 105,6,12.0,2.00
566402,8335165425,AHMAD RIFAI,17066-Akuntansi,Fakultas Ekonomi,IPK 105,20,75.5,3.77
566403,8335165445,Disya Sanova Alissa,17066-Akuntansi,Fakultas Ekonomi,IPK 105,21,75.5,3.59


In [ ]:
prodi = ['14085-HUBUNGAN MASYARAKAT','14095-PERJALANAN WISATA','15075-Teknik Elektronika','15055-Teknik Mesin','15206-Teknik Mesin','15065-Teknik Sipil','15115-Transportasi',
         '15085-TATA BOGA','15095-Tata Busana','15105-TATA RIAS','17035-Administrasi Perkantoran','17025-Manajemen Pemasaran','17045-Akuntansi','12128-MAGISTER PENDIDIKAN BAHASA INGGRIS'
         ,'17088-Magister Manajemen']

data = data[data['PROGRAM STUDI'].isin(prodi) == False]
data

,NIM,NAMA MAHASISWA,PROGRAM STUDI,FAKULTAS,SEMESTER,SKS,BOBOT,IPK
0,1101617001,RACHMAT HIDAYAT,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,150,539.3,3.59
1,1101617002,ARIF BUDI SANTOSO,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,148,553.9,3.74
2,1101617003,NURUL HISNA HERYANTO,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,159,571.3,3.59
3,1101617004,ALDINO RAMADHAN,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,148,546.7,3.69
4,1101617005,MUHAMMAD AZHAR THIFAL,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,158,593.2,3.75
...,...,...,...,...,...,...,...,...
566400,8335165396,FAUZI AKBAR EKA SAPUTRA,17066-Akuntansi,Fakultas Ekonomi,IPK 105,20,69.9,3.49
566401,8335165418,MUHAMMAD SYARIP RANGKUTI,17066-Akuntansi,Fakultas Ekonomi,IPK 105,6,12.0,2.00
566402,8335165425,AHMAD RIFAI,17066-Akuntansi,Fakultas Ekonomi,IPK 105,20,75.5,3.77
566403,8335165445,Disya Sanova Alissa,17066-Akuntansi,Fakultas Ekonomi,IPK 105,21,75.5,3.59


In [ ]:
def get_angkatan_16_17(row):
    NIM = str(row['NIM'])
    studi = row['PROGRAM STUDI'].split('-')[0]
    if studi == NIM[:len(studi)]:
        return NIM[5:7]
    else:
        return NIM[4:6]

    
data['ANGKATAN'] = data.apply(lambda row: get_angkatan_16_17(row), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
data['ANGKATAN'].value_counts()

12    53030
15    50580
11    49140
16    48692
13    47730
14    44789
17    42750
10    39470
09    36760
18    24666
19    17916
20    10123
61     3916
08       10
Name: ANGKATAN, dtype: int64

In [ ]:
data_16_17 = data[(data['ANGKATAN']=='16') | (data['ANGKATAN']=='17')]
data_16_17

,NIM,NAMA MAHASISWA,PROGRAM STUDI,FAKULTAS,SEMESTER,SKS,BOBOT,IPK,ANGKATAN
0,1101617001,RACHMAT HIDAYAT,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,150,539.3,3.59,17
1,1101617002,ARIF BUDI SANTOSO,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,148,553.9,3.74,17
2,1101617003,NURUL HISNA HERYANTO,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,159,571.3,3.59,17
3,1101617004,ALDINO RAMADHAN,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,148,546.7,3.69,17
4,1101617005,MUHAMMAD AZHAR THIFAL,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,IPK 114,158,593.2,3.75,17
...,...,...,...,...,...,...,...,...,...
566400,8335165396,FAUZI AKBAR EKA SAPUTRA,17066-Akuntansi,Fakultas Ekonomi,IPK 105,20,69.9,3.49,16
566401,8335165418,MUHAMMAD SYARIP RANGKUTI,17066-Akuntansi,Fakultas Ekonomi,IPK 105,6,12.0,2.00,16
566402,8335165425,AHMAD RIFAI,17066-Akuntansi,Fakultas Ekonomi,IPK 105,20,75.5,3.77,16
566403,8335165445,Disya Sanova Alissa,17066-Akuntansi,Fakultas Ekonomi,IPK 105,21,75.5,3.59,16


In [ ]:
list_nim_1617 = data_16_17['NIM'].unique()

In [ ]:
IPS_mahasiswa = []

for nim in list_nim_1617:
    ips = []
    data_mhs_unique = data[data['NIM']==nim]
    data_mhs_unique = data_mhs_unique.loc[::-1]
    
    SKS = data_mhs_unique['SKS'].tolist()
    BOBOT = data_mhs_unique['BOBOT'].tolist()
    IPK = data_mhs_unique['IPK'].tolist()
    
    for i in range(len(SKS)):
        if i == 0:
            ips.append(IPK[i])
        else:
            d_sks = SKS[i] - SKS[i-1]
            if d_sks == 0:
                ips.append(0)
            else:
                d_bobot = BOBOT[i] - BOBOT[i-1]
                ips.append(d_bobot/d_sks)
    
    IPS_mahasiswa.append(ips)


In [ ]:
status_mahasiswa = []

for nim in list_nim_1617:
    data_mhs_unique = data[data['NIM']==nim]
    
    SKS = data_mhs_unique['SKS'].tolist()
    
    # jika SKS 114 < 144
    sks_last = data_mhs_unique.iloc[0,:]['SKS']
    
    if sks_last < 144:
        status_mahasiswa.append(0)
        continue

    if data_mhs_unique.iloc[0]['ANGKATAN'] == '16':
      
      # jika semester 9 > semester 8
      if data_mhs_unique.iloc[2]['SKS'] < data_mhs_unique.iloc[1]['SKS']:
        status_mahasiswa.append(0)
        continue
            
    status_mahasiswa.append(1)

In [ ]:
SKS_smt1 = []
SKS_smt2 = []
SKS_smt3 = []
SKS_smt4 = []
SKS_smt5 = []
SKS_smt6 = []

for nim in list_nim_1617:
    data_mhs_unique = data[data['NIM']==nim]
    
    if data_mhs_unique.iloc[0]['ANGKATAN'] == '17':
      if len(data_mhs_unique) < 8:
        SKS_smt1.append('X')
        SKS_smt2.append('X')
        SKS_smt3.append('X')
        SKS_smt4.append('X')
        SKS_smt5.append('X')
        SKS_smt6.append('X')

        continue

      SKS_smt1.append(data_mhs_unique.iloc[7]['SKS'])
      SKS_smt2.append(data_mhs_unique.iloc[6]['SKS'])
      SKS_smt3.append(data_mhs_unique.iloc[5]['SKS'])
      SKS_smt4.append(data_mhs_unique.iloc[4]['SKS'])
      SKS_smt5.append(data_mhs_unique.iloc[3]['SKS'])
      SKS_smt6.append(data_mhs_unique.iloc[2]['SKS'])

      continue
    
    if len(data_mhs_unique) < 10:
      SKS_smt1.append('X')
      SKS_smt2.append('X')
      SKS_smt3.append('X')
      SKS_smt4.append('X')
      SKS_smt5.append('X')
      SKS_smt6.append('X')

      continue

    SKS_smt1.append(data_mhs_unique.iloc[9]['SKS'])
    SKS_smt2.append(data_mhs_unique.iloc[8]['SKS'])
    SKS_smt3.append(data_mhs_unique.iloc[7]['SKS'])
    SKS_smt4.append(data_mhs_unique.iloc[6]['SKS'])
    SKS_smt5.append(data_mhs_unique.iloc[5]['SKS'])
    SKS_smt6.append(data_mhs_unique.iloc[4]['SKS'])

In [ ]:
pd.Series(status_mahasiswa).value_counts()

0    5991
1    4225
dtype: int64

In [ ]:
list_nama = []
list_program = []
list_fakultas = []

for nim in list_nim_1617:
    data_mhs_unique = data[data['NIM']==nim]
    
    list_nama.append(data_mhs_unique.iloc[0]['NAMA MAHASISWA'])
    list_program.append(data_mhs_unique.iloc[0]['PROGRAM STUDI'])
    list_fakultas.append(data_mhs_unique.iloc[0]['FAKULTAS'])

In [ ]:
df_ips = pd.DataFrame(columns=['S'+str(i) for i in range(1, 7)])

for i in range(len(IPS_mahasiswa)):
    try:
        ips = IPS_mahasiswa[i]
        if len(IPS_mahasiswa[i]) > 6:
            ips = IPS_mahasiswa[i][:6]
        if len(ips) < 6:
            ips = ['X' for i in range(6)]

        df_ips.loc[i] = ips
    except:
        print(len(ips))

In [ ]:
df_ips

,S1,S2,S3,S4,S5,S6
0,3.51,3.63913,3.595833,3.504545,3.6,3.652381
1,3.65,3.756522,3.645833,3.704545,3.675,3.971429
2,3.53,3.569565,3.554167,3.759091,3.375,3.617391
3,3.68,3.717391,3.370833,3.890909,3.725,3.733333
4,3.68,3.73913,3.6875,3.822727,3.5875,3.882609
...,...,...,...,...,...,...
10211,3.49,3.388235,3.5,3.43,3.479167,3.3625
10212,2.0,2.7,0.0,0.0,0.0,0.0
10213,3.77,3.473684,3.427778,3.4,3.625,3.647368
10214,3.59,3.886957,3.685714,3.3,3.7,0.0


In [ ]:
new_df = pd.DataFrame()
new_df['NIM'] = list_nim_1617
new_df['NAMA'] = list_nama
new_df['PRODI'] = list_program
new_df['FAKULTAS'] = list_fakultas

for i in range(1,7):
    new_df['S'+str(i)] = df_ips['S'+str(i)]

new_df['SKS_S1'] = SKS_smt1
new_df['SKS_S2'] = SKS_smt2
new_df['SKS_S3'] = SKS_smt3
new_df['SKS_S4'] = SKS_smt4
new_df['SKS_S5'] = SKS_smt5
new_df['SKS_S6'] = SKS_smt6
new_df['STATUS'] = status_mahasiswa

new_df

,NIM,NAMA,PRODI,FAKULTAS,S1,S2,S3,S4,S5,S6,SKS_S1,SKS_S2,SKS_S3,SKS_S4,SKS_S5,SKS_S6,STATUS
0,1101617001,RACHMAT HIDAYAT,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.51,3.63913,3.595833,3.504545,3.6,3.652381,22,45,69,91,115,136,1
1,1101617002,ARIF BUDI SANTOSO,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.65,3.756522,3.645833,3.704545,3.675,3.971429,22,45,69,91,115,136,1
2,1101617003,NURUL HISNA HERYANTO,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.53,3.569565,3.554167,3.759091,3.375,3.617391,22,45,69,91,115,138,1
3,1101617004,ALDINO RAMADHAN,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.68,3.717391,3.370833,3.890909,3.725,3.733333,22,45,69,91,115,136,1
4,1101617005,MUHAMMAD AZHAR THIFAL,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.68,3.73913,3.6875,3.822727,3.5875,3.882609,22,45,69,91,115,138,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10211,8335165396,FAUZI AKBAR EKA SAPUTRA,17066-Akuntansi,Fakultas Ekonomi,3.49,3.388235,3.5,3.43,3.479167,3.3625,20,37,49,69,93,117,0
10212,8335165418,MUHAMMAD SYARIP RANGKUTI,17066-Akuntansi,Fakultas Ekonomi,2.0,2.7,0.0,0.0,0.0,0.0,6,8,8,8,8,8,0
10213,8335165425,AHMAD RIFAI,17066-Akuntansi,Fakultas Ekonomi,3.77,3.473684,3.427778,3.4,3.625,3.647368,20,39,57,80,104,123,0
10214,8335165445,Disya Sanova Alissa,17066-Akuntansi,Fakultas Ekonomi,3.59,3.886957,3.685714,3.3,3.7,0.0,21,44,65,67,71,71,0


In [ ]:
new_df = new_df[~(new_df['S1'] == 'X')]
new_df

,NIM,NAMA,PRODI,FAKULTAS,S1,S2,S3,S4,S5,S6,SKS_S1,SKS_S2,SKS_S3,SKS_S4,SKS_S5,SKS_S6,STATUS
0,1101617001,RACHMAT HIDAYAT,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.51,3.63913,3.595833,3.504545,3.6,3.652381,22,45,69,91,115,136,1
1,1101617002,ARIF BUDI SANTOSO,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.65,3.756522,3.645833,3.704545,3.675,3.971429,22,45,69,91,115,136,1
2,1101617003,NURUL HISNA HERYANTO,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.53,3.569565,3.554167,3.759091,3.375,3.617391,22,45,69,91,115,138,1
3,1101617004,ALDINO RAMADHAN,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.68,3.717391,3.370833,3.890909,3.725,3.733333,22,45,69,91,115,136,1
4,1101617005,MUHAMMAD AZHAR THIFAL,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.68,3.73913,3.6875,3.822727,3.5875,3.882609,22,45,69,91,115,138,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10211,8335165396,FAUZI AKBAR EKA SAPUTRA,17066-Akuntansi,Fakultas Ekonomi,3.49,3.388235,3.5,3.43,3.479167,3.3625,20,37,49,69,93,117,0
10212,8335165418,MUHAMMAD SYARIP RANGKUTI,17066-Akuntansi,Fakultas Ekonomi,2.0,2.7,0.0,0.0,0.0,0.0,6,8,8,8,8,8,0
10213,8335165425,AHMAD RIFAI,17066-Akuntansi,Fakultas Ekonomi,3.77,3.473684,3.427778,3.4,3.625,3.647368,20,39,57,80,104,123,0
10214,8335165445,Disya Sanova Alissa,17066-Akuntansi,Fakultas Ekonomi,3.59,3.886957,3.685714,3.3,3.7,0.0,21,44,65,67,71,71,0


In [ ]:
final_df = new_df[(new_df['S1']>0) & (new_df['S2']>0) & (new_df['S3']>0) & (new_df['S4']>0) & (new_df['S5']>0) & (new_df['S6']>0) & (new_df['SKS_S1'] != 'X') & (new_df['SKS_S2'] != 'X')]
final_df

,NIM,NAMA,PRODI,FAKULTAS,S1,S2,S3,S4,S5,S6,SKS_S1,SKS_S2,SKS_S3,SKS_S4,SKS_S5,SKS_S6,STATUS
0,1101617001,RACHMAT HIDAYAT,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.51,3.63913,3.595833,3.504545,3.6,3.652381,22,45,69,91,115,136,1
1,1101617002,ARIF BUDI SANTOSO,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.65,3.756522,3.645833,3.704545,3.675,3.971429,22,45,69,91,115,136,1
2,1101617003,NURUL HISNA HERYANTO,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.53,3.569565,3.554167,3.759091,3.375,3.617391,22,45,69,91,115,138,1
3,1101617004,ALDINO RAMADHAN,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.68,3.717391,3.370833,3.890909,3.725,3.733333,22,45,69,91,115,136,1
4,1101617005,MUHAMMAD AZHAR THIFAL,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.68,3.73913,3.6875,3.822727,3.5875,3.882609,22,45,69,91,115,138,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10208,8335165361,ILHAM ADITYA,17066-Akuntansi,Fakultas Ekonomi,3.15,3.278947,3.475,3.175,3.325,3.590476,20,39,63,87,111,132,1
10209,8335165367,MUSA FAHMUDDIN IRHAM,17066-Akuntansi,Fakultas Ekonomi,3.13,3.185714,3.25,3.5,3.58,3.26875,20,34,42,48,58,74,0
10211,8335165396,FAUZI AKBAR EKA SAPUTRA,17066-Akuntansi,Fakultas Ekonomi,3.49,3.388235,3.5,3.43,3.479167,3.3625,20,37,49,69,93,117,0
10213,8335165425,AHMAD RIFAI,17066-Akuntansi,Fakultas Ekonomi,3.77,3.473684,3.427778,3.4,3.625,3.647368,20,39,57,80,104,123,0


In [ ]:
final_df["NIM"] = final_df["NIM"].astype('str')
final_df["STATUS"] = final_df["STATUS"].astype('str')
final_df["S1"] = final_df["S1"].astype('float')
final_df["S2"] = final_df["S2"].astype('float')
final_df["S3"] = final_df["S3"].astype('float')
final_df["S4"] = final_df["S4"].astype('float')
final_df["S5"] = final_df["S5"].astype('float')
final_df["S6"] = final_df["S6"].astype('float')
final_df["SKS_S1"] = final_df["SKS_S1"].astype('float')
final_df["SKS_S2"] = final_df["SKS_S2"].astype('float')
final_df["SKS_S3"] = final_df["SKS_S3"].astype('float')
final_df["SKS_S4"] = final_df["SKS_S4"].astype('float')
final_df["SKS_S5"] = final_df["SKS_S5"].astype('float')
final_df["SKS_S6"] = final_df["SKS_S6"].astype('float')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
final_df.describe()

,S1,S2,S3,S4,S5,S6,SKS_S1,SKS_S2,SKS_S3,SKS_S4,SKS_S5,SKS_S6
count,8833.000000,8833.000000,8833.000000,8833.000000,8833.000000,8833.000000,8833.000000,8833.000000,8833.000000,8833.000000,8833.000000,8833.000000
mean,3.452967,3.484729,3.471416,3.499084,3.512775,3.618897,20.326503,41.907506,63.780143,85.684705,107.486245,128.391826
std,0.257299,0.255274,0.273366,0.272376,0.283593,0.242810,1.874045,4.154640,6.270591,8.397081,10.627582,12.574663
min,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,4.000000,10.000000,16.000000,18.000000,23.000000
25%,3.290000,3.331579,3.316667,3.340909,3.352174,3.490909,19.000000,40.000000,61.000000,83.000000,105.000000,126.000000
50%,3.480000,3.520000,3.520000,3.547619,3.566667,3.668182,21.000000,43.000000,66.000000,89.000000,111.000000,132.000000
75%,3.650000,3.678947,3.670833,3.700000,3.726087,3.795000,22.000000,45.000000,68.000000,91.000000,114.000000,136.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,23.000000,46.000000,71.000000,94.000000,119.000000,142.000000


In [ ]:
final_df = final_df[(final_df['SKS_S1'] >= 17) & (final_df['SKS_S1'] < 23) & (final_df['SKS_S2'] >= 34)]
final_df

,NIM,NAMA,PRODI,FAKULTAS,S1,S2,S3,S4,S5,S6,SKS_S1,SKS_S2,SKS_S3,SKS_S4,SKS_S5,SKS_S6,STATUS
0,1101617001,RACHMAT HIDAYAT,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.51,3.639130,3.595833,3.504545,3.600000,3.652381,22.0,45.0,69.0,91.0,115.0,136.0,1
1,1101617002,ARIF BUDI SANTOSO,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.65,3.756522,3.645833,3.704545,3.675000,3.971429,22.0,45.0,69.0,91.0,115.0,136.0,1
2,1101617003,NURUL HISNA HERYANTO,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.53,3.569565,3.554167,3.759091,3.375000,3.617391,22.0,45.0,69.0,91.0,115.0,138.0,1
3,1101617004,ALDINO RAMADHAN,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.68,3.717391,3.370833,3.890909,3.725000,3.733333,22.0,45.0,69.0,91.0,115.0,136.0,1
4,1101617005,MUHAMMAD AZHAR THIFAL,11016-Teknologi Pendidikan,Fakultas Ilmu Pendidikan,3.68,3.739130,3.687500,3.822727,3.587500,3.882609,22.0,45.0,69.0,91.0,115.0,138.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10208,8335165361,ILHAM ADITYA,17066-Akuntansi,Fakultas Ekonomi,3.15,3.278947,3.475000,3.175000,3.325000,3.590476,20.0,39.0,63.0,87.0,111.0,132.0,1
10209,8335165367,MUSA FAHMUDDIN IRHAM,17066-Akuntansi,Fakultas Ekonomi,3.13,3.185714,3.250000,3.500000,3.580000,3.268750,20.0,34.0,42.0,48.0,58.0,74.0,0
10211,8335165396,FAUZI AKBAR EKA SAPUTRA,17066-Akuntansi,Fakultas Ekonomi,3.49,3.388235,3.500000,3.430000,3.479167,3.362500,20.0,37.0,49.0,69.0,93.0,117.0,0
10213,8335165425,AHMAD RIFAI,17066-Akuntansi,Fakultas Ekonomi,3.77,3.473684,3.427778,3.400000,3.625000,3.647368,20.0,39.0,57.0,80.0,104.0,123.0,0


In [ ]:
final_df.to_excel('/content/drive/MyDrive/AdaBoost/Dataset.xlsx', index=False)